In [ ]:
import os
import sys
import random
import shutil
import time
from PIL import Image
from os import path

In [ ]:
used_prompts = []
used_answers = []
played_cards = []

players = []

In [ ]:
#initialization of file count.
number_of_prompts=0
number_of_answers=0

prompt_path=".\\Prompts"
answers_path=".\\Answers"

In [ ]:
for files in os.walk(prompt_path):
    for file in files[2]:
        number_of_prompts = number_of_prompts + 1
        
for files in os.walk(answers_path):
    for file in files[2]:
        number_of_answers = number_of_answers + 1

In [ ]:
remaining_prompts = number_of_prompts
remaining_answers = number_of_answers

to_break = False #For the switch statement

#Enter Number of Players Here
player_no = 0

#Player Selections
player_selections = {}

#Player Scores
player_scores = {}

In [ ]:
def generatePlayerCards(name):
    global number_of_answers, player_no
    if not path.exists(".\\Player Cards"):
        os.mkdir(".\\Player Cards")
    cards_per_player = min(int(number_of_answers/player_no), 10)
    player_path = ".\\Player Cards\\" + name + " Cards"
    current_card_no = 0
    if not path.exists(player_path):
        os.mkdir(player_path)
    for files in os.walk(player_path):
        for file in files[2]:
            current_card_no = current_card_no + 1
    if (current_card_no < cards_per_player):
        copyAnswerCards(player_path, (cards_per_player - current_card_no))

def copyAnswerCards(target, number):
    for i in range(number):
        current_card = ""
        while True:
            current_card = ".\\Answers\\page_" + str(random.randint(1,number_of_answers)) + ".jpg"
            if (current_card not in used_answers):
                break
        shutil.copyfile(current_card, target + "\\" + current_card.split("\\")[2])
        used_answers.append(current_card)       

def clearUsedAnswers():
    global used_answers
    used_answers = []
    
def clearUsedPrompts():
    global used_prompts
    used_prompts = []
    
def generatePrompt():
    current_prompt_path = ".\\Current Prompt"
    if path.exists(current_prompt_path):
        shutil.rmtree(current_prompt_path) 
        time.sleep(1)
    os.mkdir(current_prompt_path)
    new_prompt = ""
    while True:
            current_card = ".\\Prompts\\page_" + str(random.randint(1,number_of_prompts)) + ".jpg"
            if (current_card not in used_prompts):
                break
    shutil.copyfile(current_card, current_prompt_path + "\\" + current_card.split("\\")[2])
    used_prompts.append(current_card)
    prompt_image = Image.open(current_prompt_path + "\\" + current_card.split("\\")[2])
    prompt_image.show()
    
def clearPlayerCards(name):
    player_path = ".\\Player Cards\\" + name + " Cards"
    if path.exists(player_path):
        shutil.rmtree(player_path) 
        
def clearCurrentPrompt():
    current_prompt_path = ".\\Current Prompt"
    if path.exists(current_prompt_path):
        shutil.rmtree(current_prompt_path) 

def changePlayerNumber(number):
    global player_no
    player_no = number

def updatePlayerDetails():
    global players, player_no, player_scores
    players = []
    player_scores = {}
    print("\nEnter the number of players:")
    number = int(input())
    changePlayerNumber(number)
    for i in range(player_no):
        name = ""
        while True:
            print("\nEnter Player " + str(i + 1) + "'s Name:")
            name = str(input())
            if name in players:
                print("\nPlayer Already Exists. Enter another name.\n")
            else:
                break
        players.append(name)
        player_scores[name] = 0

def addPlayer():
    global players, player_scores
    name = ""
    while True:
        print("\nEnter Player " + str(players.len() + 1) + "'s Name:")
        name = str(input())
        if name in players:
            print("\nPlayer Already Exists. Enter another name.\n")
        else:
            break
    players.append(name)
    player_scores[name] = 0
    
def generateAllPlayerCards():
    global players
    for name in players:
        generatePlayerCards(name)

def clearAllPlayerCards():
    player_card_path = ".\\Player Cards"
    if path.exists(player_card_path):
        for files in os.walk(player_card_path):
            if files[0]!=".\\Player Cards" :
                for folder in files[2]:
                    try:
                        shutil.rmtree(files[0])
                    except: 
                        #do nothing
                        random_var = ""
        try:
            shutil.rmtree(player_card_path)
        except:
            #do nothing
            random_var = ""
        
def showCurrentPrompt():
    current_prompt_path = ".\\Current Prompt"
    for files in os.walk(current_prompt_path):
        if files[2]==[]:
            print("No prompt has been generated yet.")
            break
        for file in files[2]:
            prompt_image = Image.open(current_prompt_path + "\\" + file)
            prompt_image.show()
        
def startNewGame():
    clearPlayerScores()
    clearUsedAnswers()
    clearUsedPrompts()
    clearAllPlayerCards()
    clearCurrentPrompt()
    clearPlayedCards()
    updatePlayerDetails()
    generateAllPlayerCards()

def breakLoop():
    global to_break
    to_break = True

def generateSpecificPlayerCard():
    global players
    while True:
        print("Enter the Player's Name:")
        name = str(input())
        if name not in players:
            print("Player does not exist. Please try again.")
        else:
            generatePlayerCards(name)
            break
            
def clearPlayedCards():
    global played_cards
    played_cards = []

def makePlayerSelection():
    global player_selections, played_cards
    player_selections = {}
    for name in players:
        while True:
            print("Enter " + name + "'s Selection'")
            selection = int(input())
            path_to_file = ".\\Player Cards\\" + name + " Cards\\page_" + str(selection) + ".jpg"
            if (selection in played_cards):
                print("Card already played. Play another card.")
            elif not path.exists(path_to_file):
                print("Invalid card. Play another card.")
            else:
                player_selections[name] = selection
                played_cards.append(selection)
                os.remove(path_to_file)
                break
    
def readPlayerSelection():
    global player_selections, player_scores
    session_scores = []
    for player in player_selections:
        current_card = ".\\Answers\\page_" + str(player_selections[player]) + ".jpg"
        selection_image = Image.open(current_card)
        selection_image.show()
        returned_value = scorerFunction(player)
        session_scores.append(returned_value)
    updatePlayerScores(session_scores)
    generateAllPlayerCards()
    
def updatePlayerScores(session_scores):
    global player_scores
    current_highest_score = 0
    for name_score in session_scores:
        if(name_score[1]>current_highest_score):
            current_highest_score = name_score[1]
    for name_score in session_scores:
        if(name_score[1]==current_highest_score):
            player_scores[name_score[0]] = player_scores[name_score[0]] + 1;
    
def reshufflePlayerCards():
    clearUsedAnswers()
    clearAllPlayerCards()
    clearPlayedCards()
    time.sleep(1)
    generateAllPlayerCards()
    
def reshufflePromptCards():
    clearUsedPrompts()
    clearCurrentPrompt()
    
def scorerFunction(name):
    meme_image_path =  ".\\Score Images\\"
    print("What would you score this response? (Scale 1-5 (Bleh - Laugh out loud with inadvertent flatulence))")
    score = int(input())
    if score<1:
        score = 1
    elif score>5:
        score=5
    meme_image = meme_image_path + str(score) + ".jpg"
    meme_image = Image.open(meme_image)
    meme_image.show()
    input()
    return (name, score)

def clearPlayerScores():
    global player_scores
    player_scores = {}  
    
def viewLeaderboard():
    global player_scores
    marklist = sorted(player_scores.items(), key=lambda x:x[1])
    sortdict = dict(marklist)
    print("\n---------Leaderboard---------\n")
    for item in sortdict:
        print(item + ": " + str(sortdict[item]))

In [ ]:
## Main Menu of the Game

print("Welcome to Cards Against Humanity with the Don Bosco Choir! Hope you have lots of fun with this online game nibbas and nibbis!")

startNewGame()

switcher = {
    1: generatePrompt,
    2: showCurrentPrompt,
    3: generateAllPlayerCards,
    4: generateSpecificPlayerCard,
    5: clearCurrentPrompt,
    6: clearAllPlayerCards,
    7: makePlayerSelection,
    8: readPlayerSelection,
    9: reshufflePlayerCards,
    10: reshufflePromptCards,
    11: viewLeaderboard,
    12: startNewGame,
    13: breakLoop
}

while True:
    print("\nSelect an option from the below choices:")
    print("1. Generate New Prompt\n2. Show Current Prompt Again\n3. Generate All Player Cards\n4. Generate Specific Player Cards\n5. Clear Current Prompt\n6. Clear All Player Cards\n7. Make Player Selections\n8. Read Player Selections and Score\n9. Reshuffle Player Cards\n10. Reshuffle Prompt Cards\n11. View Leaderboard\n12. Create a New Game\n13. Exit")
    choice = int(input())
    to_break = False            
    if(choice < 1 or choice > 13):
        print("You entered an invalid choice. Please try again.")
    else:
        switcher.get(choice)()
    if(to_break):
        break

print("Thank you for playing this game! Hope to have many more games like this in the future!")